## Implementing Iterators, Iterables and Collections

### Iterables and Iterators
* iterators decouple how we retrieve items from an object from the structure of the object
  + the items might be generated, stored and managed by different structures and mechanisms, such as collection, gnerator, file or sensor data from streaming
  + the retrieval of the items is managed by another type of closely related objects called an iterator
    + the iterator encapsulates the current position in the iterable and has has an iterface that allows us to get the next value and to determine when the underlying iterable is exhausted
    + iterators in Python is known as forward iterators, meaning that they traverse the underlying iterable in a forward direction one item at a time, without jumpping and changing direction
    + the underlying series of elements may be conceptually infinite or unending if the results are produced dynamically by a computation or a sensor rather than retrieved from storage
    + Python also provides reverse iterator, which traverse the items from the end to the beginning.
      + only apply to collections with a finite number of elements with a definite end
* As a quick review of how iterator and iterable works, the following code showed the 3 typical steps to obtain and advance an iterator and get data from an iterable:

```python

# obtain an iterator from iterable
iterator = iter(iterable)

# traverse the iterable to retrieve data by next(iterator)
try:
    item = next(iterator)
    print(item)

# handle the end of the iterable
except StopIteration:
    print("No more items")
    
```    


* Two ways of creating iterables
  + instantiating built-in collections
  + using generators either by generator functions or expressions